In [1]:
# to access functions from root directory
import sys
sys.path.append('/data/ad181/RemoteDir/multilevel_ppo')

In [2]:
%matplotlib notebook
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from tqdm.notebook import trange, tqdm

import gym
from stable_baselines3.ppo import PPO, MlpPolicy
from stable_baselines3.ppo_multi_level import PPO_ML
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env.subproc_vec_multi_level_env import SubprocVecMultiLevelEnv
from stable_baselines3.common.envs.multi_level_ressim_env import MultiLevelRessimEnv
from stable_baselines3.common.logger import configure

from utils.custom_eval_callback import CustomEvalCallback, CustomEvalCallbackParallel
from utils.plot_functions import plot_learning
from utils.env_evaluate_functions import eval_actions

In [3]:
seed=1
case='mlmc_analysis'
data_dir='./data'
log_dir='./data/'+case

In [4]:
os.makedirs(data_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [5]:
with open('../envs_params/env_data_v1/env_train_dict.pkl', 'rb') as input:
    env_ck_dict = pickle.load(input)

In [6]:
# generate dictionaries for env (env_dict_), n_steps (T_ml) and minibatch (M_ml) for `n_level`s
n_levels=2
fine_level = len(env_ck_dict)
env_dict_ = {}
for i,l in enumerate(range(fine_level-n_levels, fine_level)):
    print(i+1,'->',l+1)
    env_dict_[i+1] = env_ck_dict[l+1]

1 -> 3
2 -> 4


In [7]:
for seed in range(1,2):
    if True: 
        print(f'seed {seed}')
        log_dir = './data/'+case+'/seed_'+str(seed)
        os.makedirs(log_dir, exist_ok=True)
        T = {1:100, 2:100} # n_steps
        N = 50 # number of actors
        M = {1:500, 2:500} # minibatch size
        I = 1200 # number of iterations
        K = 20 # number of epochs
        
        log_interval = I/120
        
        fine_level = len(env_dict_)
        
        print('generate callback ...')
        eval_callback = CustomEvalCallback( env_dict_[fine_level], 
                                            best_model_save_path=None, 
                                            n_eval_episodes=1,
                                            log_path=str(log_dir)+'/results_eval', 
                                            eval_freq=log_interval*T[1]  )
        
        print('vectorize environment ...')
        
        # generate PPO_ML parameters for MLMC analysis. 
        # we choose same n_steps and batch_size values on levels because only fine level values are used in the analysis
        env_dict = {}
        n_steps_dict = {}
        batch_size_dict = {}
        for env, level in zip(env_dict_.values(), env_dict_.keys()):
            print(f"vectorize env level {level}")
            env_dict[level] = make_vec_env( MultiLevelRessimEnv, 
                                    n_envs=N, 
                                    seed=seed, 
                                    env_kwargs= {"ressim_params":env.ressim_params, "level":env.level}, 
                                    vec_env_cls=SubprocVecMultiLevelEnv )
            n_steps_dict[level] = T[level]
            batch_size_dict[level] = M[level]
        
        print(env_dict_[level].observation_space)
        print('model definition ..')
        model = PPO_ML(policy=MlpPolicy,
                           env=env_dict,
                           learning_rate = 1e-6,
                           n_steps = n_steps_dict,
                           batch_size = batch_size_dict,
                           n_epochs = K,
                           clip_range = 0.1,
                           ent_coef = 0.001,
                           vf_coef = 0.5,
                           policy_kwargs = dict(net_arch=[70,70,50], log_std_init=-2.9),
                           verbose = 1,
                           seed = seed,
                           target_kl = 0.05,
                           device = "auto")
        # set logger for the model
        new_logger = configure(log_dir)
        model.set_logger(new_logger)
        print('policy learning and analysis ..')
        e2 = [1e-2, 1e-3, 1e-4]
        analysis_interval = I/10
        model.mlmc_analysis(total_timesteps=N*T[1]*I, 
                            n_expt= 100000, 
                            eps_array=np.sqrt(e2),
                            log_interval=log_interval,
                            analysis_interval=analysis_interval, 
                            step_comp_time_dict={1:0.15,2:1.0})
        model.save(log_dir+'/PPO', exclude=['env_dict'])
        del model
        for level in env_dict.keys():
            env_dict[level].close()


seed 1
generate callback ...
vectorize environment ...
vectorize env level 1
vectorize env level 2
Box(-1.0, 1.0, (35,), float64)
model definition ..
Using cuda device
Logging to ./data/mlmc_analysis/seed_1
policy learning and analysis ..
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 5             |
|    ep_rew_mean          | 0.677         |
| time/                   |               |
|    fps                  | 58            |
|    iterations           | 10            |
|    time_elapsed         | 854           |
|    total_timesteps      | 50000         |
| train/                  |               |
|    approx_kl            | 1.4137933e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | 31.1          |
|    explained_variance   | -7.21         |
|    learning_rate        | 1e-06         |
|    loss                 | 0.244         |
|    n_update

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 5             |
|    ep_rew_mean          | 0.683         |
| time/                   |               |
|    fps                  | 61            |
|    iterations           | 100           |
|    time_elapsed         | 8157          |
|    total_timesteps      | 500000        |
| train/                  |               |
|    approx_kl            | 1.0966796e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | 31.1          |
|    explained_variance   | -3.36         |
|    learning_rate        | 1e-06         |
|    loss                 | 0.135         |
|    n_updates            | 1980          |
|    policy_gradient_loss | -0.000207     |
|    std                  | 0.055         |
|    value_loss           | 0.202         |
-------------------------------------------
--------------------------------

100%|██████████| 2000/2000 [32:48<00:00,  1.02it/s]

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 5             |
|    ep_rew_mean          | 0.682         |
| time/                   |               |
|    fps                  | 51            |
|    iterations           | 120           |
|    time_elapsed         | 11645         |
|    total_timesteps      | 600000        |
| train/                  |               |
|    approx_kl            | 1.3106818e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | 31.1          |
|    explained_variance   | -2.94         |
|    learning_rate        | 1e-06         |
|    loss                 | 0.115         |
|    n_updates            | 2380          |
|    policy_gradient_loss | -0.000255     |
|    std                  | 0.055         |
|    value_loss           | 0.183         |
-------------------------------------------


analysis of MLMC estimator for 100000 number of experimets...
------------------------------------------------------------------
| expt_results/           |                                      |
|    C_l                  | [0.15, 1.0]                          |
|    N                    | 100000                               |
|    P_l                  | [0.0614, 0.0611]                     |
|    V_l                  | [0.99, 0.98]                         |
| mc_results/             |                                      |
|    C_mc                 | 1                                    |
|    N_mc                 | [197, 1962, 19615]                   |
|    P_mc                 | [0.0246, 0.0873, 0.0529]             |
|    V_mc                 | 0.98                                 |
|    eps_mc               | [0.1        0.03162278 0.01      ]   |
| mlmc_results/           |                                      |
|    C_ml                 | [[0.15, 1.15], [0.15, 1.15], [0.1... |


...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 5             |
|    ep_rew_mean          | 0.714         |
| time/                   |               |
|    fps                  | 53            |
|    iterations           | 1200          |
|    time_elapsed         | 112474        |
|    total_timesteps      | 6000000       |
| train/                  |               |
|    approx_kl            | 2.3973094e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.1           |
|    entropy_loss         | 31.1          |
|    explained_variance   | 0.643         |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0387        |
|    n_updates            | 23980         |
|    policy_gradient_loss | -0.000289     |
|    std                  | 0.055         |
|    value_loss           | 0.0169        |
-------------------------------------------


analysis of MLMC estimator for 100000 number of experimets...


/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
